# Age Detection Model

**Import necessary libraries and prepare the dataset for operations**

In [1]:
import os                  
import cv2                 
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import matplotlib.pyplot as plt


Total images loaded: 23709


**Load the dataset**

In [7]:
image_data=r'C:\Users\lenovo\OneDrive\Desktop\jupyter_Project\age_detect\dataset'      
image_size = 224            # We resize all images to 224x224 (needed for MobileNetV2)
X,y=[],[]

for file in os.listdir(image_data):
    try:
        age=int(file.split("_")[0])
        img_path=os.path.join(image_data,file)
        image=cv2.imread(img_path)
        X.append(image)
        y.append(image)
    except:
        continue

X=np.array(X)
y=np.array(y)

print(f"Total images loaded: {len(X)}")

Total images loaded: 23709


**Split the data for training and testing**

In [2]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
#print the quantity of training and testing data
print(f"Training data: {len(X_train)}")
print(f"Testing data: {len(X_test)}")

Training data: 18967
Testing data: 4742


**Build the MobileNetV2-Based Model**

In [6]:
#Define the shape for input
input_shape=(224,224,3)

## Load pre-trained MobileNetV2 model without the top layer to make our own layer
base_model=MobileNetV2(
    input_shape=input_shape,
    include_top=False,
    weights='imagenet'
    )

#Stop the base model layer
base_model.trainable=False

# create the new model on first of base
inputs=Input(shape=input_shape)
x=base_model(inputs,training=False)
x=GlobalAveragePooling2D()(x)
x=Dropout(0.3)(x)
x=Dense(128,activation='relu')(x)
x=Dropout(0.3)(x)

output=Dense(1)(x) #One output neuraon for predicting the age

#Bulid the proper Model
model=Model(inputs,output)

#compile the model using the MSL(mean square loss) which is part of regression
model.compile(optimizer='adam',
             loss='mean-squared_error',
             metrics=['mae'])
#Mean absolute Error to creating more readable
print("done")

done


**Training the model**

In [8]:
#set the Training Callbacks
checkpoint=ModelCheckpoint('age_model.h5',      # Save better accuracy model to this .h5 file
                          monitor='val_loss',   # Checking the validation loss to save model
                          save_best_only=True,
                          verbose=1)
